# Challenge Part 2
## Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [25]:
%matplotlib inline

# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import scipy.stats as sts
# Import mpl to change the plot configurations using rcParams.
import matplotlib as mpl

import random 
import timeit
import requests
import time
import gmaps

#Notice the difference in import syntax for these modules. This eliminates the need to double type the name
#For example: datetime.datetime.xyz
from citipy import citipy
from datetime import datetime
from scipy.stats import linregress

# Import API keys.
from config import weather_api_key

from config import g_key
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Part 2 section 2 ####################################################################################
# Load in csv 
city_weatherDB_to_load = os.path.join("weather_data", "WeatherPy_database.csv")

city_weather_df = pd.read_csv(city_weatherDB_to_load)

In [26]:
city_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
0,0,Cruz Alta,-28.64,-53.61,43.41,89,0,12.28,BR,2020-06-28 01:49:27,clear sky,0.00,0.0
1,1,Hobart,-42.88,147.33,48.20,70,0,8.05,AU,2020-06-28 01:49:28,clear sky,0.00,0.0
2,2,Tuatapere,-46.13,167.68,48.56,93,100,5.06,NZ,2020-06-28 01:49:28,light rain,0.58,0.0
3,3,Lebu,-37.62,-73.65,50.92,91,100,19.13,CL,2020-06-28 01:49:28,moderate rain,2.07,0.0
4,4,Tasiilaq,65.61,-37.64,39.20,93,100,4.70,GL,2020-06-28 01:49:28,light rain,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,566,Paamiut,61.99,-49.67,40.69,87,100,2.51,GL,2020-06-28 01:49:57,overcast clouds,0.00,0.0
567,567,Hirara,24.80,125.28,86.00,79,75,14.99,JP,2020-06-28 01:49:57,broken clouds,0.00,0.0
568,568,Chiang Klang,19.29,100.86,82.40,88,40,1.57,TH,2020-06-28 01:49:57,scattered clouds,0.00,0.0
569,569,Puerto Lempira,15.27,-83.77,79.30,90,100,6.67,HN,2020-06-28 01:49:57,light rain,0.50,0.0


In [27]:
city_weather_df.drop(city_weather_df.columns[[0]], axis=1, inplace=True)

In [28]:
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
0,Cruz Alta,-28.64,-53.61,43.41,89,0,12.28,BR,2020-06-28 01:49:27,clear sky,0.00,0.0
1,Hobart,-42.88,147.33,48.20,70,0,8.05,AU,2020-06-28 01:49:28,clear sky,0.00,0.0
2,Tuatapere,-46.13,167.68,48.56,93,100,5.06,NZ,2020-06-28 01:49:28,light rain,0.58,0.0
3,Lebu,-37.62,-73.65,50.92,91,100,19.13,CL,2020-06-28 01:49:28,moderate rain,2.07,0.0
4,Tasiilaq,65.61,-37.64,39.20,93,100,4.70,GL,2020-06-28 01:49:28,light rain,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
566,Paamiut,61.99,-49.67,40.69,87,100,2.51,GL,2020-06-28 01:49:57,overcast clouds,0.00,0.0
567,Hirara,24.80,125.28,86.00,79,75,14.99,JP,2020-06-28 01:49:57,broken clouds,0.00,0.0
568,Chiang Klang,19.29,100.86,82.40,88,40,1.57,TH,2020-06-28 01:49:57,scattered clouds,0.00,0.0
569,Puerto Lempira,15.27,-83.77,79.30,90,100,6.67,HN,2020-06-28 01:49:57,light rain,0.50,0.0


In [29]:
# Part 2 section 3 ####################################################################################
# Ask the customer prompts
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


rain_toggle = str(input("Do you want it to be raining? (yes/no) "))
snow_toggle = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [30]:
#Filter by temperature range
preferred_cities_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= max_temp) & \
                                       (city_weather_df["Max Temp"] >= min_temp)]

preferred_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
8,Ixtapa,20.70,-105.20,86.00,70,75,6.93,MX,2020-06-28 01:49:28,broken clouds,0.00,0.0
24,Kapaa,22.08,-159.32,84.20,66,20,14.99,US,2020-06-28 01:49:29,few clouds,0.00,0.0
26,Khatanga,71.97,102.50,77.79,39,100,6.04,RU,2020-06-28 01:49:29,overcast clouds,0.00,0.0
27,Faanui,-16.48,-151.75,76.68,78,97,18.19,PF,2020-06-28 01:49:29,overcast clouds,0.00,0.0
28,Manavalakurichi,8.13,77.30,78.80,88,75,3.36,IN,2020-06-28 01:49:29,mist,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
562,Devils Lake,48.11,-98.87,87.80,45,1,12.75,US,2020-06-28 01:45:29,clear sky,0.00,0.0
563,Ternate,0.80,127.40,82.13,73,100,4.50,ID,2020-06-28 01:45:54,light rain,0.41,0.0
567,Hirara,24.80,125.28,86.00,79,75,14.99,JP,2020-06-28 01:49:57,broken clouds,0.00,0.0
568,Chiang Klang,19.29,100.86,82.40,88,40,1.57,TH,2020-06-28 01:49:57,scattered clouds,0.00,0.0


In [31]:
#Filter by RAIN (filtering on the prefered cities generated byt he temerature filter)
if rain_toggle == "no":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain last hr"] == 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain last hr"] != 0)]

preferred_cities_df     

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
8,Ixtapa,20.70,-105.20,86.00,70,75,6.93,MX,2020-06-28 01:49:28,broken clouds,0.0,0.0
24,Kapaa,22.08,-159.32,84.20,66,20,14.99,US,2020-06-28 01:49:29,few clouds,0.0,0.0
26,Khatanga,71.97,102.50,77.79,39,100,6.04,RU,2020-06-28 01:49:29,overcast clouds,0.0,0.0
27,Faanui,-16.48,-151.75,76.68,78,97,18.19,PF,2020-06-28 01:49:29,overcast clouds,0.0,0.0
28,Manavalakurichi,8.13,77.30,78.80,88,75,3.36,IN,2020-06-28 01:49:29,mist,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
556,Lahat,-3.80,103.53,81.09,72,26,1.48,ID,2020-06-28 01:49:56,scattered clouds,0.0,0.0
561,Lashio,22.93,97.75,76.71,86,100,0.63,MM,2020-06-28 01:49:57,overcast clouds,0.0,0.0
562,Devils Lake,48.11,-98.87,87.80,45,1,12.75,US,2020-06-28 01:45:29,clear sky,0.0,0.0
567,Hirara,24.80,125.28,86.00,79,75,14.99,JP,2020-06-28 01:49:57,broken clouds,0.0,0.0


In [32]:
#Filter by SNOW (filtering on the prefered cities generated by the temerature & rain filters)
if snow_toggle == "no":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow last hr"] == 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow last hr"] != 0)]

preferred_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
8,Ixtapa,20.70,-105.20,86.00,70,75,6.93,MX,2020-06-28 01:49:28,broken clouds,0.0,0.0
24,Kapaa,22.08,-159.32,84.20,66,20,14.99,US,2020-06-28 01:49:29,few clouds,0.0,0.0
26,Khatanga,71.97,102.50,77.79,39,100,6.04,RU,2020-06-28 01:49:29,overcast clouds,0.0,0.0
27,Faanui,-16.48,-151.75,76.68,78,97,18.19,PF,2020-06-28 01:49:29,overcast clouds,0.0,0.0
28,Manavalakurichi,8.13,77.30,78.80,88,75,3.36,IN,2020-06-28 01:49:29,mist,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
556,Lahat,-3.80,103.53,81.09,72,26,1.48,ID,2020-06-28 01:49:56,scattered clouds,0.0,0.0
561,Lashio,22.93,97.75,76.71,86,100,0.63,MM,2020-06-28 01:49:57,overcast clouds,0.0,0.0
562,Devils Lake,48.11,-98.87,87.80,45,1,12.75,US,2020-06-28 01:45:29,clear sky,0.0,0.0
567,Hirara,24.80,125.28,86.00,79,75,14.99,JP,2020-06-28 01:49:57,broken clouds,0.0,0.0


In [9]:
preferred_cities_df.shape

(146, 12)

In [10]:
# Part 2 sections 4,5,6 ####################################################################################

In [33]:
#Generate hotel dataframe - hotel column will populate through a later API call
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
8,Ixtapa,MX,86.00,20.70,-105.20,broken clouds,
24,Kapaa,US,84.20,22.08,-159.32,few clouds,
26,Khatanga,RU,77.79,71.97,102.50,overcast clouds,
27,Faanui,PF,76.68,-16.48,-151.75,overcast clouds,
28,Manavalakurichi,IN,78.80,8.13,77.30,mist,
31,Buala,SB,83.61,-8.14,159.59,overcast clouds,
33,Avarua,CK,75.20,-21.21,-159.78,clear sky,
34,Lazaro Cardenas,MX,81.61,17.96,-102.20,overcast clouds,
36,Sosua,DO,82.99,19.77,-70.52,overcast clouds,
42,Dongying,CN,89.78,37.46,118.49,broken clouds,


In [34]:
hotel_df.shape

(164, 7)

In [35]:
# Set API parameters to search for a hotel.

# 5000 is in meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [20]:
# Iterate through the DataFrame for each lat & lon
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
8,Ixtapa,MX,86.00,20.70,-105.20,broken clouds,
24,Kapaa,US,84.20,22.08,-159.32,few clouds,
26,Khatanga,RU,77.79,71.97,102.50,overcast clouds,
27,Faanui,PF,76.68,-16.48,-151.75,overcast clouds,
28,Manavalakurichi,IN,78.80,8.13,77.30,mist,
...,...,...,...,...,...,...,...
556,Lahat,ID,81.09,-3.80,103.53,scattered clouds,
561,Lashio,MM,76.71,22.93,97.75,overcast clouds,
562,Devils Lake,US,87.80,48.11,-98.87,clear sky,
567,Hirara,JP,86.00,24.80,125.28,broken clouds,


In [37]:
hotel_df.count()

City           164
Country        164
Max Temp       164
Lat            164
Lng            164
Description    164
Hotel Name     164
dtype: int64

In [38]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
#Edited code with map markers
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# The pop-up box uses html formatting
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [40]:
# Updated map with pop-up added
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))